In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.svm import SVR

In [ ]:
data_train = pd.read_csv('CUP_IT_train_data_new.csv')
data_test =  pd.read_csv('CUP_IT_test_data1.csv', sep=';')

In [ ]:
data_train['cu_education_level']=data_train.cu_education_level.fillna('03. Высшее')
data_train['cc_balance'] = data_train.cc_balance.fillna(1667.7381963197197).apply(np.abs)
data_train['cl_balance'] = data_train.cl_balance.fillna(0).apply(np.abs)
data_train['ml_balance'] = data_train.ml_balance.fillna(0).apply(np.abs)
data_train['pl_balance'] = data_train.pl_balance.fillna(0).apply(np.abs)
data_train['td_volume'] = data_train.td_volume.fillna(0).apply(np.abs)
data_train['ca_volume'] = data_train.ca_volume.fillna(9.374111562519905).apply(np.abs)
data_train['sa_volume'] = data_train.sa_volume.fillna(0).apply(np.abs)
data_train['mf_volume'] = data_train.mf_volume.fillna(0).apply(np.abs)

In [ ]:
data_train.loc[data_train.big_city == 'MSK', 'big_city'] = 0
data_train.loc[data_train.big_city == 'MLN', 'big_city'] = 1
data_train.loc[data_train.big_city == 'SPB', 'big_city'] = 2
data_train.loc[data_train.big_city == 'OTH', 'big_city'] = 3
data_train['cu_education_level'] = data_train['cu_education_level'].apply(lambda x: int(str(x)[1]))
data_train[['big_city']] = data_train[['big_city']].astype(int)
data_train['dlk_cob_date'] = pd.to_datetime(data_train.dlk_cob_date).dt.month

In [ ]:
data_train_drop = data_train.drop(['gi_smooth_3m',
                              'cu_empl_area',
                              'cu_empl_level',
                              'cur_quantity_cc',
                              'cur_quantity_deposits',
                              'cur_quantity_dc',
                              'cur_quantity_accounts',
                              'cur_quantity_saccounts',
                              'cur_quantity_mf',
                              'cc_balance',
                              'cl_balance',
                              'ml_balance',
                              'pl_balance',
                              'td_volume',
                              'ca_volume',
                              'mf_volume',
                              'dc_cash_spend_v',
                              'cc_cash_spend_v',
                              'dc_pos_spend_v',
                              'cc_pos_spend_v',
                              'cur_qnt_sms',
                              'cu_empl_cur_dur_m'], axis = 1)

In [ ]:
X = data_train_drop.drop(['cif_id', 'dlk_cob_date', 'active'], axis = 1)
y = data_train_drop.active
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state = 241)

In [ ]:
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=40, learning_rate=0.2)
gbm.fit(X_train, y_train)

In [ ]:
def isstr(x):
    if isinstance(x,str):
        return np.nan
    else:
        return float(x)
data_test['cu_education_level']=data_test.cu_education_level.fillna('03. Высшее')
data_test['cc_balance'] = data_test.cc_balance.dropna().apply(isstr).fillna(1034.043421753243).apply(np.abs)
data_test['cl_balance'] = data_test.cl_balance.dropna().apply(isstr).fillna(0).apply(np.abs)
data_test['ml_balance'] = data_test.ml_balance.dropna().apply(isstr).fillna(0).apply(np.abs)
data_test['pl_balance'] = data_test.pl_balance.dropna().apply(isstr).fillna(0).apply(np.abs)
data_test['td_volume'] = data_test.td_volume.fillna(0)
data_test['ca_volume'] = data_test.ca_volume.fillna(0)
data_test['sa_volume'] = data_test.sa_volume.apply(isstr).fillna(0)
data_test['mf_volume'] = data_test.mf_volume.fillna(0)

In [ ]:
data_test.loc[data_test.big_city == 'MSK', 'big_city'] = 0
data_test.loc[data_test.big_city == 'MLN', 'big_city'] = 1
data_test.loc[data_test.big_city == 'SPB', 'big_city'] = 2
data_test.loc[data_test.big_city == 'OTH', 'big_city'] = 3
data_test['cu_education_level'] = data_test['cu_education_level'].apply(lambda x: int(str(x)[1]))
data_test[['big_city']] = data_test[['big_city']].astype(int)
data_test['dlk_cob_date'] = pd.to_datetime(data_test.dlk_cob_date).dt.month

In [ ]:
data_test_prof = data_test.drop(['gi_smooth_3m',
                              'cu_empl_area',
                              'cu_empl_level',
                              'cur_quantity_cc',
                              'cur_quantity_deposits',
                              'cur_quantity_dc',
                              'cur_quantity_accounts',
                              'cur_quantity_saccounts',
                              'cur_quantity_mf',
                              'cc_balance',
                              'cl_balance',
                              'ml_balance',
                              'pl_balance',
                              'td_volume',
                              'ca_volume',
                              'mf_volume',
                              'dc_cash_spend_v',
                              'cc_cash_spend_v',
                              'dc_pos_spend_v',
                              'cc_pos_spend_v',
                              'cur_qnt_sms',
                              'cu_empl_cur_dur_m'], axis = 1)

In [ ]:
X_test_data = data_test_prof.drop(['cif_id', 'dlk_cob_date', 'active'], axis = 1)

In [ ]:
proba_test = gbm.predict_proba(X_test_data)
data_test = data_test.merge(pd.DataFrame(list(map(lambda x: x[1], proba_test))), 
                              right_index=True, 
                              left_index = True).rename(columns = {0: 'proba'})

In [ ]:
data_test_date_prob = data_test[['cif_id','dlk_cob_date', 'proba']]
data_prob = pd.DataFrame(data_test_date_prob.cif_id.unique()).rename(columns = {0: 'cif_id'})
for month in sorted(data_test_date_prob.dlk_cob_date.unique()):
    data_prob = data_prob.merge(data_test_date_prob.loc[data_test_date_prob.dlk_cob_date == month]\
                                          .drop(['dlk_cob_date'],axis=1).rename(columns = {'proba': month})
                                          , how = 'outer')
data_prob = data_prob.merge(data_test_date_prob.loc[data_test_date_prob.dlk_cob_date == 12]\
                                          .drop(['dlk_cob_date'],axis=1).rename(columns = {'gi_smooth_3m': 12})
                                          , how = 'outer')
data_prob = data_prob.rename(columns = {'proba': 12})

In [ ]:
def isstr_1(x):
    if str.isalpha(x[0]) or str.isalpha(x[len(x)-1]) or str.isalpha(x[len(x)//2]) or str.isalpha(x[len(x)//2+1]):
        return np.nan
    else:
        return x
#Form to predict date of drop
data_test_date_prof = data_test[['cif_id',
                              'dlk_cob_date', 
                              'gi_smooth_3m']]
data_test_date_prof['gi_smooth_3m'] = data_test_date_prof.gi_smooth_3m.apply(str)
data_test_date_prof['gi_smooth_3m'] = data_test_date_prof.gi_smooth_3m.apply(isstr_1)
data_prof = pd.DataFrame(data_test_date_prof.cif_id.unique()).rename(columns = {0: 'cif_id'})
for month in sorted(data_test_date_prof.dlk_cob_date.unique()):
    data_prof = data_prof.merge(data_test_date_prof.loc[data_test_date_prof.dlk_cob_date == month]\
                                          .drop(['dlk_cob_date'],axis=1).rename(columns = {'gi_smooth_3m': month})
                                          , how = 'outer')
data_prof = data_prof.merge(data_test_date_prof.loc[data_test_date_prof.dlk_cob_date == 12]\
                                          .drop(['dlk_cob_date'],axis=1).rename(columns = {'gi_smooth_3m': 12})
                                          , how = 'outer')

In [ ]:
def fill(x):
    index = 12
    line = x
    while index >= 0 and np.isnan(x[index]):
        index -= 1
    if index >= 6:
        line[:index] = line[:index].fillna(line[1:].mean())
        return line
data_prob = data_prob.apply(fill,axis = 1)

In [ ]:
clf = SVR(kernel='linear', C=0.045, epsilon = 0.08899)
for i in range(len(data_prob)):
    line = data_prob.drop(['cif_id'],axis = 1).iloc[i].dropna()
    clf.fit(pd.DataFrame(list(range(1, len(line)+1))), line)
    data_prob.iloc[i, len(line):] = tuple(clf.predict(pd.DataFrame(list(range(len(line),13)))))
    print(data_prob.iloc[i])

In [ ]:
data_prof = data_prof.dropna(subset = list(range(1,13)))
clf = SVR(kernel='linear', C=0.045, epsilon = 0.08899)
for i in range(len(data_prof)):
    line = data_prof.drop(['cif_id'],axis = 1).iloc[i].dropna()
    clf.fit(pd.DataFrame(list(range(1, len(line)+1))), line)
    data_prof.iloc[i, len(line):] = tuple(clf.predict(pd.DataFrame(list(range(len(line),13)))))
    print(data_prof.iloc[i])

In [ ]:
list_of_prof = []
for i in range(len(data_prof)):
    flag = False
    month = 6
    while month<12 and data_prob.iloc[i,month] >= 0.68:
        month += 1
    while month < 12:
        data_prof.iloc[i,month]=0
        month += 1
    s = data_prof.iloc[i, 7:].sum()
    list_of_prof.append(s)
df = pd.DataFrame(data_test_date_prof.cif_id.unique()).rename(columns = {0: 'cif_id'})
df = df.merge(pd.DataFrame(list_of_prof),right_index=True, 
                              left_index = True, how = 'outer').rename(columns = {0: 'profit_for_6_month'}).fillna(0)
df.to_csv('Predict_profit.csv')